<h1 align="left">Vision To Text Experimentation</h1>

<p align="left">
  ITESM
  
  <a href="https://www.linkedin.com/in/juanrtato/">Juan Ricardo Albarracin B.</a>
  <br>
  <a href="">Luis Ángel Oporto Añacato.</a>
  <br>
  <a href="">David Alexis García Espinosa.</a>
  <br>
  <b>Last updated:</b> <i>19/05/2025</i>
  <br><br>
  <a target="_blank">
    <img src="https://github.com/QData/TextAttack/workflows/Github%20PyTest/badge.svg" alt="Testing">
  </a>
  <a href="https://img.shields.io/badge/version-0.1.0-blue.svg?cacheSeconds=2592000">
    <img src="https://img.shields.io/badge/version-0.1.0-blue.svg?cacheSeconds=2592000" alt="Version" height="18">
  </a>
</p><br>

In [1]:
CONFIG_PATH = '../datalake/config_vtt.json'
MODEL_PATH = '../datalake/TSVIT/best.pth'
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts')))

In [2]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
import ast
import json
import torch
from tsvit import torch_utils, model_architecture
from visiontotext import visiontotextmodel
from pastis24 import get_dataloaders

/home/juanrtato/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

2.7.0+cu126
12.6
True


In [5]:
device_ids = [0]
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)
device = torch_utils.get_device(device_ids, allow_cpu=False)
with open("../datalake/label_names_en.json", "r") as json_file:
    label_names_en = json.load(json_file)
with open("../datalake/colormap.txt", "r") as txt_file:
    colormap = txt_file.readlines()
colormap = [ast.literal_eval(line.strip().rstrip(',')) for line in colormap]

In [6]:
records = []
dataloaders_vtt = get_dataloaders(config)

Loading PASTIS2SEQUENCE dataset...
PASTIS2SEQUENCE dataset loaded!


In [7]:
num_epochs = 2
encoder = model_architecture.get_model(config, device)
encoder.load_state_dict(torch.load(MODEL_PATH))
encoder.eval()
print(encoder)

TSViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b t c (h p1) (w p2) -> (b h w) t (p1 p2 c)', p1=2, p2=2)
    (1): Linear(in_features=40, out_features=128, bias=True)
  )
  (to_temporal_embedding_input): Linear(in_features=366, out_features=128, bias=True)
  (temporal_transformer): Transformer(
    (layers): ModuleList(
      (0-3): 4 x ModuleList(
        (0): PreNorm(
          (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (fn): Attention(
            (to_qkv): Linear(in_features=128, out_features=384, bias=False)
            (to_out): Sequential(
              (0): Linear(in_features=128, out_features=128, bias=True)
              (1): Dropout(p=0.0, inplace=False)
            )
          )
        )
        (1): PreNorm(
          (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (fn): FeedForward(
            (net): Sequential(
              (0): Linear(in_features=128, out_features=512, bias=True)
              (1): GE

In [8]:
model_vtt = visiontotextmodel.VisionToTextModel(encoder, decoder_model="gpt2", input_dim=128).to(device)
optimizer = torch.optim.Adam(model_vtt.parameters(), lr=1e-4)
model_vtt.train()

Decoder input size: 768


VisionToTextModel(
  (encoder): TSViT(
    (to_patch_embedding): Sequential(
      (0): Rearrange('b t c (h p1) (w p2) -> (b h w) t (p1 p2 c)', p1=2, p2=2)
      (1): Linear(in_features=40, out_features=128, bias=True)
    )
    (to_temporal_embedding_input): Linear(in_features=366, out_features=128, bias=True)
    (temporal_transformer): Transformer(
      (layers): ModuleList(
        (0-3): 4 x ModuleList(
          (0): PreNorm(
            (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (fn): Attention(
              (to_qkv): Linear(in_features=128, out_features=384, bias=False)
              (to_out): Sequential(
                (0): Linear(in_features=128, out_features=128, bias=True)
                (1): Dropout(p=0.0, inplace=False)
              )
            )
          )
          (1): PreNorm(
            (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (fn): FeedForward(
              (net): Sequential(
                (

## Without training

In [9]:
def generate_caption(model, input_single, prompt="An image of", max_length=30):
    model.eval()
    model = model.to(device)
    input_single = input_single.to(device)

    with torch.no_grad():
        visual_emb = model.encoder.get_encoder_embeddings(input_single)
        print('visual_emb shape: ', visual_emb.shape)

        #visual_cls = visual_emb.mean(dim=1, keepdim=True)
        #visual_cls = visual_emb[:, :, :]
        visual_cls = visual_emb[:, 0, :]
        #print('visual_cls shape: ', visual_emb[:, -1, :].shape)
        visual_proj = model.projector(visual_cls).unsqueeze(1)
        print('visual_proj shape: ', visual_proj.shape)

        input_ids = model.tokenizer(prompt, return_tensors="pt", padding=True).input_ids.to(device)
        input_ids = input_ids.expand(visual_cls.shape[0], -1)
        text_emb = model.decoder.transformer.wte(input_ids)
        print('text_emb shape: ', text_emb.shape)

        input_emb = torch.cat([visual_proj, text_emb], dim=1)

        # Construir attention_mask
        visual_attention_mask = torch.ones(visual_proj.size()[:-1], dtype=torch.long).to(device)  # [B, 1]
        text_attention_mask = (input_ids != model.tokenizer.pad_token_id).long()  # [B, T]
        attention_mask = torch.cat([visual_attention_mask, text_attention_mask], dim=1)  # [B, 1 + T]

        # Definir pad_token_id
        pad_token_id = model.tokenizer.pad_token_id
        if pad_token_id is None:
            pad_token_id = model.tokenizer.eos_token_id

        outputs = model.decoder.generate(
            inputs_embeds=input_emb,
            attention_mask=attention_mask,
            max_length=max_length,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=1.0,
            num_return_sequences=1,
            pad_token_id=pad_token_id
        )

    return model.tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "The image shows"
sample_dict, texts, img_path = next(iter(dataloaders_vtt['eval']))
image_sequence = sample_dict['inputs'].to(device)[1].unsqueeze(0)
caption = generate_caption(model_vtt, image_sequence, prompt, max_length=50)
caption

Input shape: torch.Size([1, 60, 24, 24, 11])
Permuted Input shape: torch.Size([1, 60, 11, 24, 24])
Input temporal encoder shape: torch.Size([60, 366])
Temporal position embedding shape: torch.Size([1, 60, 128])
After patch embedding shape: torch.Size([144, 60, 128])
After reshaping: torch.Size([1, 144, 60, 128])
After adding temporal position embedding shape: torch.Size([1, 144, 60, 128])
After reshaping: torch.Size([144, 60, 128])
CLS temporal tokens shape: torch.Size([144, 19, 128])
After concatenation shape (dim 1): torch.Size([144, 79, 128])
After temporal transformer shape: torch.Size([144, 79, 128])
After spatial transformer shape: torch.Size([19, 144, 128])
After mean pooling shape (dim 1): torch.Size([19, 128])
Final output shape: torch.Size([1, 19, 128])
visual_emb shape:  torch.Size([1, 19, 128])
visual_proj shape:  torch.Size([1, 1, 768])
text_emb shape:  torch.Size([1, 3, 768])


' a top-level application. It was called AOS/AOS:P/AOS/AOS from 2.0.10, and it was named App.\n\nThere was no way to use AOS on'

## Model Training

In [10]:
def training_step(model, batch, optimizer, device):
    model.train()
    sample_dict, texts, img_path = batch
    image_sequence = sample_dict['inputs'].to(device)
    print(f"Image shape:, {image_sequence.shape}")            # [B, C, H, W] o similar               # [B, T]
    tokenizer = model.tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
    input_ids = tokenizer.input_ids
    print(f"Input ids shape:, {input_ids.shape}")            # [B, T]

    loss, _ = model(image_sequence, input_ids)
    if optimizer is not None:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return loss.item()

In [11]:
dataloaders_vtt.keys()

dict_keys(['train', 'eval'])

In [12]:
def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for sample in dataloader:
            loss = training_step(model, sample, optimizer=None, device=device)  # sin backprop
            total_loss += loss
    return total_loss / len(dataloader)

for epoch in range(1):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    model_vtt.train()
    for sample in dataloaders_vtt['train']:
        loss = training_step(model_vtt, sample, optimizer, device)
        print(f"Train Loss: {loss:.4f}")
    
    val_loss = evaluate(model_vtt, dataloaders_vtt['eval'], device)
    print(f"Eval Loss: {val_loss:.4f}")


Epoch 1/2
Image shape:, torch.Size([24, 60, 24, 24, 11])
Input ids shape:, torch.Size([24, 65])
Input shape: torch.Size([24, 60, 24, 24, 11])
Permuted Input shape: torch.Size([24, 60, 11, 24, 24])
Input temporal encoder shape: torch.Size([1440, 366])
Temporal position embedding shape: torch.Size([24, 60, 128])
After patch embedding shape: torch.Size([3456, 60, 128])
After reshaping: torch.Size([24, 144, 60, 128])
After adding temporal position embedding shape: torch.Size([24, 144, 60, 128])
After reshaping: torch.Size([3456, 60, 128])
CLS temporal tokens shape: torch.Size([3456, 19, 128])
After concatenation shape (dim 1): torch.Size([3456, 79, 128])
After temporal transformer shape: torch.Size([3456, 79, 128])
After spatial transformer shape: torch.Size([456, 144, 128])
After mean pooling shape (dim 1): torch.Size([456, 128])
Final output shape: torch.Size([24, 19, 128])
visual_proj: torch.Size([24, 1, 768]), text_emb: torch.Size([24, 65, 768])


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Train Loss: 7.6776
Image shape:, torch.Size([24, 60, 24, 24, 11])
Input ids shape:, torch.Size([24, 67])
Input shape: torch.Size([24, 60, 24, 24, 11])
Permuted Input shape: torch.Size([24, 60, 11, 24, 24])
Input temporal encoder shape: torch.Size([1440, 366])
Temporal position embedding shape: torch.Size([24, 60, 128])
After patch embedding shape: torch.Size([3456, 60, 128])
After reshaping: torch.Size([24, 144, 60, 128])
After adding temporal position embedding shape: torch.Size([24, 144, 60, 128])
After reshaping: torch.Size([3456, 60, 128])
CLS temporal tokens shape: torch.Size([3456, 19, 128])
After concatenation shape (dim 1): torch.Size([3456, 79, 128])
After temporal transformer shape: torch.Size([3456, 79, 128])
After spatial transformer shape: torch.Size([456, 144, 128])
After mean pooling shape (dim 1): torch.Size([456, 128])
Final output shape: torch.Size([24, 19, 128])
visual_proj: torch.Size([24, 1, 768]), text_emb: torch.Size([24, 67, 768])
Train Loss: 4.3310
Image shape:

In [13]:
def generate_caption(model, input_single, prompt="An image of", max_length=30):
    model.eval()
    model = model.to(device)
    input_single = input_single.to(device)

    with torch.no_grad():
        visual_emb = model.encoder.get_encoder_embeddings(input_single)
        print('visual_emb shape: ', visual_emb.shape)

        #visual_cls = visual_emb.mean(dim=1, keepdim=True)
        #visual_cls = visual_emb[:, :, :]
        visual_cls = visual_emb[:, 0, :]
        #print('visual_cls shape: ', visual_emb[:, -1, :].shape)
        visual_proj = model.projector(visual_cls).unsqueeze(1)
        print('visual_proj shape: ', visual_proj.shape)

        input_ids = model.tokenizer(prompt, return_tensors="pt", padding=True).input_ids.to(device)
        input_ids = input_ids.expand(visual_cls.shape[0], -1)
        text_emb = model.decoder.transformer.wte(input_ids)
        print('text_emb shape: ', text_emb.shape)

        input_emb = torch.cat([visual_proj, text_emb], dim=1)

        # Construir attention_mask
        visual_attention_mask = torch.ones(visual_proj.size()[:-1], dtype=torch.long).to(device)  # [B, 1]
        text_attention_mask = (input_ids != model.tokenizer.pad_token_id).long()  # [B, T]
        attention_mask = torch.cat([visual_attention_mask, text_attention_mask], dim=1)  # [B, 1 + T]

        # Definir pad_token_id
        pad_token_id = model.tokenizer.pad_token_id
        if pad_token_id is None:
            pad_token_id = model.tokenizer.eos_token_id

        outputs = model.decoder.generate(
            inputs_embeds=input_emb,
            attention_mask=attention_mask,
            max_length=max_length,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=1.0,
            num_return_sequences=1,
            pad_token_id=pad_token_id
        )

    return model.tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "The image shows"
sample_dict, texts, img_path = next(iter(dataloaders_vtt['eval']))
image_sequence = sample_dict['inputs'].to(device)[1].unsqueeze(0)
caption = generate_caption(model_vtt, image_sequence, prompt, max_length=50)
prompt + caption

Input shape: torch.Size([1, 60, 24, 24, 11])
Permuted Input shape: torch.Size([1, 60, 11, 24, 24])
Input temporal encoder shape: torch.Size([60, 366])
Temporal position embedding shape: torch.Size([1, 60, 128])
After patch embedding shape: torch.Size([144, 60, 128])
After reshaping: torch.Size([1, 144, 60, 128])
After adding temporal position embedding shape: torch.Size([1, 144, 60, 128])
After reshaping: torch.Size([144, 60, 128])
CLS temporal tokens shape: torch.Size([144, 19, 128])
After concatenation shape (dim 1): torch.Size([144, 79, 128])
After temporal transformer shape: torch.Size([144, 79, 128])
After spatial transformer shape: torch.Size([19, 144, 128])
After mean pooling shape (dim 1): torch.Size([19, 128])
Final output shape: torch.Size([1, 19, 128])
visual_emb shape:  torch.Size([1, 19, 128])
visual_proj shape:  torch.Size([1, 1, 768])
text_emb shape:  torch.Size([1, 3, 768])


'The image shows a cultivated area of grapevine spanning approximately 11300 m2, which represents a significant percentage of the total agricultural area: 57600 m2.\n'

In [ ]:
#torch.save(model_vtt, '../datalake/model_vtt_full.pth')

## With training

To test the last trained model:

In [17]:
# Y para cargar:
model = torch.load('../datalake/model_vtt_full.pth', weights_only=False)
model.to(device)
model.eval()

VisionToTextModel(
  (encoder): TSViT(
    (to_patch_embedding): Sequential(
      (0): Rearrange('b t c (h p1) (w p2) -> (b h w) t (p1 p2 c)', p1=2, p2=2)
      (1): Linear(in_features=40, out_features=128, bias=True)
    )
    (to_temporal_embedding_input): Linear(in_features=366, out_features=128, bias=True)
    (temporal_transformer): Transformer(
      (layers): ModuleList(
        (0-3): 4 x ModuleList(
          (0): PreNorm(
            (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (fn): Attention(
              (to_qkv): Linear(in_features=128, out_features=384, bias=False)
              (to_out): Sequential(
                (0): Linear(in_features=128, out_features=128, bias=True)
                (1): Dropout(p=0.0, inplace=False)
              )
            )
          )
          (1): PreNorm(
            (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (fn): FeedForward(
              (net): Sequential(
                (

In [18]:
caption = generate_caption(model, image_sequence, prompt, max_length=50)

Input shape: torch.Size([1, 60, 24, 24, 11])
Permuted Input shape: torch.Size([1, 60, 11, 24, 24])
Input temporal encoder shape: torch.Size([60, 366])
Temporal position embedding shape: torch.Size([1, 60, 128])
After patch embedding shape: torch.Size([144, 60, 128])
After reshaping: torch.Size([1, 144, 60, 128])
After adding temporal position embedding shape: torch.Size([1, 144, 60, 128])
After reshaping: torch.Size([144, 60, 128])
CLS temporal tokens shape: torch.Size([144, 19, 128])
After concatenation shape (dim 1): torch.Size([144, 79, 128])
After temporal transformer shape: torch.Size([144, 79, 128])
After spatial transformer shape: torch.Size([19, 144, 128])
After mean pooling shape (dim 1): torch.Size([19, 128])
Final output shape: torch.Size([1, 19, 128])
visual_emb shape:  torch.Size([1, 19, 128])
visual_proj shape:  torch.Size([1, 1, 768])
text_emb shape:  torch.Size([1, 3, 768])


In [19]:
prompt + caption

'The image shows the presence of grapevine with a dimension close to 27800 m2, orchard with an approximate area of 14200 m2, which constitutes an important part of the agricultural area in the region: 57600 m2.'